### ganache

### 1. 개발

In [1]:
%%writefile ../src/FallbackTest.sol
// SPDX-License-Identifier: MIT
pragma solidity >=0.6.0;

contract FallbackTest {
    event PrintLog(string);
    
    function callA() pure public returns(string memory) {
        return "doing callA";
    }
    fallback() external {
        emit PrintLog("fallback called");
    }
}

Writing ../src/FallbackTest.sol


### 2. 컴파일 with combined-json

In [2]:
!solc ../src/FallbackTest.sol --combined-json abi > ../src/FallbackTestABI.json

In [4]:
!type ..\src\FallbackTestABI.json

{"contracts":{"../src/FallbackTest.sol:FallbackTest":{"abi":"[{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"stateMutability\":\"nonpayable\",\"type\":\"fallback\"},{\"inputs\":[],\"name\":\"callA\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"pure\",\"type\":\"function\"}]"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


In [5]:
!solc ../src/FallbackTest.sol --combined-json bin > ../src/FallbackTestBIN.json

In [7]:
!type ..\src\FallbackTestBIN.json

{"contracts":{"../src/FallbackTest.sol:FallbackTest":{"bin":"608060405234801561001057600080fd5b50610190806100206000396000f3fe608060405234801561001057600080fd5b506004361061002f5760003560e01c8063e7f09e051461009a57610030565b5b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100a261011d565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100e25780820151818401526020810190506100c7565b50505050905090810190601f16801561010f5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60606040518060400160405280600b81526020017f646f696e672063616c6c4100000000000000000000000000000000000000000081525090509056fea26469706673582212201359d3af22064a2bd3bcdec83f048f415b04b53698aa08d0f01b966003c00ade64736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


### 3. 배포

In [10]:
%%writefile ../src/FallbackTestDeployFromFile.js
var Web3 = require('web3');
var _abiJson = require('../src/FallbackTestABI.json');
var _binJson = require('../src/FallbackTestBIN.json');

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractName = Object.keys(_abiJson.contracts);
console.log("- contract name: "+contractName[0]);
var _abi = JSON.parse(_abiJson.contracts[contractName].abi);
var _bin = _binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from "+accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data:_bin})
        .send({from:accounts[0], gas:1000000}, function(err, transactionHash) {
            if (!err)
                console.log("hash: "+transactionHash);
        });
    console.log("The contract deployed to: "+deployed.options.address)
}
deploy()

Overwriting ../src/FallbackTestDeployFromFile.js


In [11]:
!node ../src/FallbackTestDeployFromFile.js

- contract name: ../src/FallbackTest.sol:FallbackTest
Deploying the contract from 0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de
hash: 0xfaaa5affd1f97a6afad2219d84e7cbe2bb6ccfc00541dc659169c0bb09352416
The contract deployed to: 0x58D1b899a3d483a85cA4C2e02Ef78c5fC10B9D8d


### 4. 사용

In [17]:
%%writefile ../src/FallbackTestUseFromFile.js
var Web3 = require('web3');
var _abiJson = require('../src/FallbackTestABI.json');

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}

contractName = Object.keys(_abiJson.contracts);
console.log("- contract name: "+contractName[0]);
var _abi = JSON.parse(_abiJson.contracts[contractName].abi);

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("\nAccount: "+accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance Before: "+balanceBefore);
    
    var _contract = new web3.eth.Contract(_abi, "0x58D1b899a3d483a85cA4C2e02Ef78c5fC10B9D8d");
    var event = _contract.events.PrintLog(function (err, ret) {
        if (!err)
            console.log("\n\nEvent Fired: "+JSON.stringify(ret.returnValues));
    });
    
    _contract.methods.callA().call().then(console.log);
    
    // fallback called
    web3.eth.sendTransaction({from:accounts[0], to:"0x5FA96D98362f92cc2E3dAF1F0fB53E2E717e4368"})
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("\nBalance After: "+balanceAfter);
    console.log("Balance diff: "+(balanceBefore - balanceAfter));
    
    process.exit(1);
}
doIt()

Overwriting ../src/FallbackTestUseFromFile.js


In [18]:
!node ../src/FallbackTestUseFromFile.js

- contract name: ../src/FallbackTest.sol:FallbackTest

Account: 0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de
Balance Before: 99983143599999998740
doing callA

Balance After: 99983143599999998740
Balance diff: 0
